In [ ]:
!pip install papermill
!pip install scrapbook
!pip install cornac
!pip install retrying
!pip install pandera

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/microsoft/recommenders.git

Cloning into 'recommenders'...
remote: Enumerating objects: 37381, done.
remote: Counting objects: 100% (913/913), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 37381 (delta 622), reused 761 (delta 581), pack-reused 36468
Receiving objects: 100% (37381/37381), 205.54 MiB | 19.84 MiB/s, done.
Resolving deltas: 100% (25264/25264), done.


In [ ]:
!mv /content/recommenders /content/recommender
!cp -r /content/recommender/recommenders /content/

In [ ]:
!pwd

/content


In [ ]:
import sys
import os
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from sklearn.preprocessing import LabelEncoder

print("System version: {}".format(sys.version))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.10.6 (main, May 29 2023, 11:10:38) [GCC 11.3.0]
Cornac version: 1.16


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
test = pd.read_csv('./Dataset/test.csv')
train = pd.read_csv('./Dataset/train.csv')

In [ ]:
test = test.drop('xd', axis=1)
train = train.drop('xd', axis=1)

In [ ]:
train_r = train.groupby(['qid','article_id'])['response'].agg(response = sum)
test_r = test.groupby(['qid','article_id'])['response'].agg(response = sum)

In [ ]:
train = train_r.index.to_frame(index = False)
test = test_r.index.to_frame(index = False)

In [ ]:
train['click_count'] = train_r.values
test['click_count'] = test_r.values

In [ ]:
train = train[['qid','article_id','click_count']]
test = test[['qid','article_id','click_count']]

In [ ]:
test = test.rename(columns={'qid': 'userID', 'article_id': 'itemID', 'click_count': 'rating'})
train = train.rename(columns={'qid': 'userID', 'article_id': 'itemID', 'click_count': 'rating'})

In [ ]:
test = test[["userID", "itemID", "rating"]]
train = train[["userID", "itemID", "rating"]]

In [ ]:
test.head()

,userID,itemID,rating
0,85837,1320,0
1,85837,1561,0
2,85837,2247,0
3,85837,3755,0
4,85837,3792,0


In [ ]:
train.head()

,userID,itemID,rating
0,85837,1073,0
1,85837,1226,0
2,85837,1561,1
3,85837,2050,0
4,85837,2105,0


In [ ]:
df_articles = pd.read_csv('./Dataset/articles.csv')

In [ ]:
df_articles.head()

,Unnamed: 0,article_id,title,topic_des_2,product_des_2,topic_des_1,product_des_1,headline,headlinemedium,headlineshort,teaser,teasershort,teaserlong,text
0,0,7387,How and why to build a bond ladder,['Investing in Bonds'],['Bonds'],"['Investing strategies', 'Portfolio strategies']","['Bond Ladder Tool', 'Bonds', 'Fixed income', ...",How and why to build a bond ladder,How to build a bond ladder,Need steady income?,Ladders may offer predictable income and rate ...,"Even in uncertain rate environments, bond ladd...",Staggering bond maturities may help to create ...,Key takeaways * Staggering bond maturities ...
1,1,1561,Inheriting IRAs from someone other than your s...,['Estate Planning'],['Inherited IRA'],"['Asset withdrawals', 'Beneficiaries', 'Distri...","['IRA', 'Inherited IRA', 'RMD Calculator', 'Re...",Inheriting IRAs from someone other than your s...,Nonspousal inherited IRA rules,Nonspousal inherited IRA rules,Understand the new distribution rules for inhe...,Understand the new distribution rules for inhe...,Understand the new distribution rules for inhe...,Key takeaways * The SECURE Act changed the ...
2,2,5326,Do you really need a bank?,['Financial Planning'],"['Brokerage', 'CDs', 'Cash Management', 'Credi...","['Banking', 'Debt management', 'Saving and spe...",NaN,Do you really need a bank?,Do you really need a bank?,Do you really need a bank?,The answer is not as clear-cut as it once was....,Banks aren't the only game in town when it com...,Mobile banking has made the brick-and-mortar b...,Key takeaways | -----------------------------...
3,3,9893,Beware of cashing out a 401(k),['Saving for Retirement'],"['401(k)', 'IRA', 'Retirement Accounts', 'Roll...","['Investing strategies', 'Retirement', 'Tax ef...","['401(k)', 'IRA', 'Retirement accounts', 'Roll...",Beware of cashing out a 401(k),Beware of cashing out a 401(k),Beware of cashing out,Evaluate all of your options before cashing ou...,Evaluate all of your options before cashing ou...,It might be tempting to withdraw some money wh...,Key takeaways * There can be an immediate c...
4,4,2601,Smart strategies for required distributions,"['Living in Retirement', 'Required Minimum Dis...",['IRA'],"['Asset withdrawals', 'Charitable giving', 'Di...","['529 plans', 'Cash Management Account', 'IRA'...",Smart strategies for required distributions,Smart IRA withdrawal strategies,Smart IRA withdrawal strategies,Answer 4 key questions to help you manage RMD ...,Answer 4 questions to help you use required mi...,See 4 key questions to answer to help you mana...,Key takeaways * You may not need your RMDs ...


In [ ]:
df_articles['all_text'] = pd.Series(df_articles[['headline','teaser','text']].fillna('').values.tolist()).str.join(' ')
df_articles = df_articles[['article_id','all_text']]
df_articles = df_articles.rename(columns={'article_id': 'itemID'})

In [ ]:
df_articles

,itemID,all_text
0,7387,How and why to build a bond ladder Ladders may...
1,1561,Inheriting IRAs from someone other than your s...
2,5326,Do you really need a bank? The answer is not a...
3,9893,Beware of cashing out a 401(k) Evaluate all of...
4,2601,Smart strategies for required distributions An...
...,...,...
66,2394,Tax moves to consider by year-end Seize the ch...
67,1226,"Q1 market update: Uneven, mature expansion Eas..."
68,5473,Q1 2020 sector scorecard A healthy outlook for...
69,2852,7 credit card tips Make credit cards work for ...


In [ ]:
tcm_id_le = LabelEncoder()

df_articles['itemID'] = tcm_id_le.fit_transform(df_articles['itemID'])
test['itemID'] = tcm_id_le.transform(test['itemID'])
train['itemID'] = tcm_id_le.transform(train['itemID'])

In [ ]:
df_articles

,itemID,all_text
0,50,How and why to build a bond ladder Ladders may...
1,4,Inheriting IRAs from someone other than your s...
2,35,Do you really need a bank? The answer is not a...
3,67,Beware of cashing out a 401(k) Evaluate all of...
4,12,Smart strategies for required distributions An...
...,...,...
66,11,Tax moves to consider by year-end Seize the ch...
67,1,"Q1 market update: Uneven, mature expansion Eas..."
68,38,Q1 2020 sector scorecard A healthy outlook for...
69,14,7 credit card tips Make credit cards work for ...


In [ ]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 463
Number of items: 66


# Models

In [ ]:
# Instantiate the recommender models to be compared
gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=10,
    learner="adam",
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf1 = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf2 = cornac.models.NeuMF(
    name="NeuMF_pretrained",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    num_factors=gmf.num_factors,
    layers=mlp.layers,
    act_fn=mlp.act_fn,
).pretrain(gmf, mlp)


In [ ]:
# # UserKNN methods
# user_knn_cosine = cornac.models.UserKNN(k=K, similarity="cosine", name="UserKNN-Cosine")
# user_knn_pearson = cornac.models.UserKNN(
#     k=K, similarity="pearson", name="UserKNN-Pearson"
# )

bpr = neumf1

In [ ]:
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/10 [00:00<?, ?it/s]

Took 142.4402 seconds for training.


In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 0.2712 seconds for prediction.


In [ ]:
all_predictions.head()

,userID,itemID,prediction
19645,85837,20,0.014378
19646,85837,24,0.016020
19647,85837,32,0.018991
19648,85837,43,0.015730
19649,85837,50,0.016867


In [ ]:
k = 10
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=k)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=k)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.240389
NDCG:	0.668287
Precision@K:	0.623758
Recall@K:	0.298960
